# EV_charginging_facts

***Table of Contents***
1. Ev Car Poulation 
    - Information about the dataset
    - Extracting 
        1. Saving to a df
        2. Extracting the columns that we need and saving as a new df 
        3. Removing null values that we do not need 
    - Transforming 
        1. Adding registiration date column
        2. Splitting the vehichle location to lat and long 
        3. Adding country column:
        4. Adding current registiraction status
        5. Adding quantity field
        6. Adding pk field 
        7. Adding SCD columns
        8. Transforming column names 
2. EV Stations Data-Set
    - Information about the dataset
    - Extracting 
        1. Saving to a df
        2. Extracting the columns that we need and saving as a new df 
        3. Removing null values and filtering out unnecessary date that we do not need 
    - Transforming 
        1. Transforming status code to understandable format for users <br>
        2. Adding the quantity column<br>
        3. Extracting dollor amount from the price column and transforming to float<br>
        4. Adding currency column<br>
        5. Adding PK column<br>
        6. Adding SK Column <br>
        7. Transforming column names <br>
        8. Transforming activity_date column to date type <br>
        9. Transforming column names <br>
3.  Combining two data set to create the ev_charging_fact table
4.  Creating the Location data frame 
    a. extracting location info and save as a df
    b. populate region info from a look up table
    c. create a pk column
    e. merge to the result df to bring pk column to the fact df
5.  Creating the date data frame 
6.  Creating and loading location dim
7.  Creating and Loading date dim
8.  Creating and loading ev_charging dim
9.  Creating and loading ev pop dim
10. Creatin and loading ev-charging_fact table


***Libraries***

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import  pandas as pd 

## 1  EV Car Population Data 

Information about the data set

* last updated at March 18, 2023
The VIN is assigned to a specific vehicle model by the manufacturer, and it remains with the vehicle throughout its lifetime. In contrast, the DOL Vehicle ID is assigned by the Washington State Department of Licensing (DOL) to a specific vehicle when it is registered and titled in Washington State. So the VIN identifies the make, model, and features of a particular vehicle, while the DOL Vehicle ID identifies the vehicle as registered in the state of Washington

we need to think this table as scd 3 because the records that this table can hardly change, for example VIN or vehicle id these identifications are important and they almost never change but only field that can change is the registiration status. In our data we do not have field for it but we can create one and assume that we have this field to keep things simple. Logic would be when we refreshing we will check
* do we have the record of the specific car in our new data
    * if we do we skip it 
    * else we set the current reg status not_active and prev status to active 


* total populated and extracted data fields :19

### 1.1 ---------------------extracting the data -----------

1. Loading the data set in a data frame

In [4]:
ev_pop = pd.read_csv('data/Electric_Vehicle_Population_Data.csv')
ev_pop.head(2)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJ3E1EB2J,Suffolk,Suffolk,VA,23435.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,NaN,476647986,POINT (-76.42443 36.8752),NaN,5.180008e+10
1,5YJ3E1ECXL,Yakima,Yakima,WA,98908.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,14.0,103490145,POINT (-120.56916 46.58514),PACIFICORP,5.307700e+10


2 Exploring: we see that we have 121978 rows  and 17 columns

In [5]:
ev_pop.shape

(121978, 17)

In [6]:
ev_pop.columns

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract'],
      dtype='object')

3. extracting the information that we need and saving to a different data frame 

In [7]:
df_ev_pop = ev_pop[['VIN (1-10)', 'DOL Vehicle ID', 'Make','Model','Vehicle Location', 'City', 'State', 'Model Year']]

4. Removing Null Values 

In [8]:
df_ev_pop = df_ev_pop.loc[df_ev_pop['City'].notnull()]
df_ev_pop = df_ev_pop.loc[df_ev_pop['City'].notna()]

### 1.2 ------------------- Transforming the Data ---------

**1. adding registiration date column:** (this field does not exsist but we need it because we are missing a date dimension) and we are basing registiration date from model year. we are naming this field as activity date because we are going to use this field for our merge <br>


In [9]:
df_ev_pop["activity_date"] = "01/01/" + df_ev_pop['Model Year'].astype(str)

now we need to change the data type ad date type

In [10]:
df_ev_pop["activity_date"]= df_ev_pop["activity_date"].astype('datetime64[D]')

let's drop make 

we can now drop model year since we have it as a date 

In [11]:
df_ev_pop = df_ev_pop.drop(columns=['Model Year'])

**2. Splitting the vehichle location to lat and long**: We need these fields to spearated so we can create viz or run some analytics queries and we need to make sure 
that they are stored as float incase we need to calculate the distance in between two location

In [12]:
df_ev_pop[["val", "Latitude", "Longitude"]] = df_ev_pop['Vehicle Location'].str.split(' ', expand=True)
# removing paranthesis 
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].str[1:]
df_ev_pop["Longitude"] =df_ev_pop["Longitude"].str.replace(')',"")
df_ev_pop["Longitude"]= df_ev_pop["Longitude"].str.strip()
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].str.strip()
df_ev_pop.head()

,VIN (1-10),DOL Vehicle ID,Make,Model,Vehicle Location,City,State,activity_date,val,Latitude,Longitude
0,5YJ3E1EB2J,476647986,TESLA,MODEL 3,POINT (-76.42443 36.8752),Suffolk,VA,2018-01-01,POINT,-76.42443,36.8752
1,5YJ3E1ECXL,103490145,TESLA,MODEL 3,POINT (-120.56916 46.58514),Yakima,WA,2020-01-01,POINT,-120.56916,46.58514
2,WA1LAAGE7M,144941534,AUDI,E-TRON,POINT (-120.56916 46.58514),Yakima,WA,2021-01-01,POINT,-120.56916,46.58514
3,5YJ3E1EA1K,168513922,TESLA,MODEL 3,POINT (-79.4172 36.58598),Danville,VA,2019-01-01,POINT,-79.4172,36.58598
4,1FADP5CU9E,150749378,FORD,C-MAX,POINT (-76.21549 36.92478),Norfolk,VA,2014-01-01,POINT,-76.21549,36.92478


In [13]:
df_ev_pop["Longitude"]= df_ev_pop["Longitude"].astype(float)
df_ev_pop["Latitude"] =  df_ev_pop["Latitude"].astype(float)

In [14]:
df_ev_pop = df_ev_pop.drop(columns=['val'])
df_ev_pop = df_ev_pop.drop(columns=['Vehicle Location'])

**3. Adding country column:** We need this field for the merge and it is missing in the original data. We know that all this locations are in united states. so we populate all the rows with US

In [15]:
df_ev_pop["country"]= 'US'

**4. Adding current registiraction status :** this field does not exsist but we know that people move and when they move the registiration to that state might move to another state and you need to move your registiration. Therefore we are creating this field to keep track of this change

In [16]:
df_ev_pop["current_reg_status"]= 'active'

**5. Adding quantity field:** we want to keep track how many cars in that region so we are adding the quantity field

In [17]:
df_ev_pop["quantity"]= int(1)

**6. Adding pk field :** I checked to see if the the records are unique and all of theme were. I used DOL Vehicle ID to check. Since we know we have unique cars we can create new pk number for the cars. 

In [18]:
df_ev_pop =df_ev_pop.reset_index(drop=True)
df_ev_pop['ev_car_pop_id'] =  df_ev_pop.index+1

**7. Adding SCD columns:** Since we know that we do not need to keep track of anything other than the registiration status we can add scd type 3 columns.We are adding prev_reg_status column to keep track of the activty status and we will add the updated_at column to see when we update the registiration status.

In [19]:
df_ev_pop["prev_reg_status"] = ''

We can use current date to show the date that acitivity date has been adjusted. We will have registiration date and we will have updated at so this will give you the enough historical information about the this table

In [20]:
df_ev_pop["reg_sta_updated_at"]=  pd.Series(dtype='datetime64[ns]')

8. Transforming column names : some of the columns have empty spots and we also want to show that this dv-pop data so we add 

In [21]:
df_ev_pop.columns

Index(['VIN (1-10)', 'DOL Vehicle ID', 'Make', 'Model', 'City', 'State',
       'activity_date', 'Latitude', 'Longitude', 'country',
       'current_reg_status', 'quantity', 'ev_car_pop_id', 'prev_reg_status',
       'reg_sta_updated_at'],
      dtype='object')

In [22]:
df_ev_pop.rename(columns={'DOL Vehicle ID': 'dol_vehicle_id', 
                          'VIN (1-10)': 'vehicle_vin', 
                          'Make': 'vehicle_make',
                          'Latitude': 'latitude',
                          'Longitude': 'longitude',
                          'City': 'city',
                          'State': 'state',
                          'Model': 'vehicle_model',
                          'quantity': 'vehicle_pop_quantity',
                          }, inplace=True)

In [23]:
df_ev_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121976 entries, 0 to 121975
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   vehicle_vin           121976 non-null  object        
 1   dol_vehicle_id        121976 non-null  int64         
 2   vehicle_make          121976 non-null  object        
 3   vehicle_model         121872 non-null  object        
 4   city                  121976 non-null  object        
 5   state                 121976 non-null  object        
 6   activity_date         121976 non-null  datetime64[ns]
 7   latitude              121946 non-null  float64       
 8   longitude             121946 non-null  float64       
 9   country               121976 non-null  object        
 10  current_reg_status    121976 non-null  object        
 11  vehicle_pop_quantity  121976 non-null  int64         
 12  ev_car_pop_id         121976 non-null  int64         
 13 

our final data siz

In [24]:
df_ev_pop.shape

(121976, 15)

-----------------------

# 2 ev_station data set 

* "Access Code": This tells us if the station is public or private
* "Status Code": 

### 2. extracting the data -------------------------

**1. Saving to a df:**

In [25]:
ev_station = pd.read_csv('data/alt_fuel_stations.csv')
ev_station.head(5)

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,MD,NaN,NaN,NaN
1,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,NaN
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,Q,3000,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,HD,NaN,NaN,NaN
4,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN,NaN


In [26]:
ev_station.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

**2. Extracting the columns that we need and saving as a new df:**

In [27]:
ev_station_v1 = ev_station[['ID','Status Code', "Station Name", 'Fuel Type Code', "Access Code",  "City", "State","Country", "Latitude", "Longitude", "Open Date", 'EV Pricing'] ]


In [28]:
len(ev_station_v1)

56800

**3. Removing null values and not nessary rows to limit unnessary data:**

In [29]:
ev_station_v1 = ev_station_v1.loc[ev_station_v1['City'].notnull()]
ev_station_v1 = ev_station_v1.loc[ev_station_v1['City'].notna()]

In [30]:
ev_station_v1.head()

,ID,Status Code,Station Name,Fuel Type Code,Access Code,City,State,Country,Latitude,Longitude,Open Date,EV Pricing
0,17,E,Spire - Montgomery Operations Center,CNG,private,Montgomery,AL,US,32.367916,-86.267021,2010-12-01,NaN
1,42,E,PS Energy - Atlanta,CNG,public,Atlanta,GA,US,33.745843,-84.398837,1994-07-15,NaN
2,45,E,Metropolitan Atlanta Rapid Transit Authority,CNG,private,Atlanta,GA,US,33.821911,-84.367461,1996-12-15,NaN
3,64,E,United Parcel Service,CNG,private,Atlanta,GA,US,33.760256,-84.543822,1997-01-01,NaN
4,72,E,Clean Energy - Texas Department of Transportation,CNG,public,Houston,TX,US,29.779918,-95.435385,1996-12-15,NaN


fuel type cng = Compressed Natural Gas (CNG) is a gasoline and diesel fuel alternative consisting primarily of methane. The gas is associated with other fossil fuels (coal or oil) and is created by methanogenic organisms in landfill

E85= E85 (or flex fuel) is a term that refers to high-level ethanol-gasoline blends containing 51% to 83% ethanol, depending on geography and season (see Fuel Properties and E85 Flex Fuel Specification).

ELEC : electric 

LPG= LNG is natural gas in its liquid form. 

BD: ?

HY : Hydrogen Fueling

In [31]:
ev_station_v1["Fuel Type Code"].unique()

array(['CNG', 'E85', 'ELEC', 'LNG', 'LPG', 'BD', 'HY'], dtype=object)

since we are interested in ev chars we do not need other charging stations information, we filter them out and then drop the fuel type code 

In [32]:
ev_station_v1 = ev_station_v1.loc[ev_station_v1["Fuel Type Code"]=='ELEC']
# ev_station_v3 = ev_station_v2.loc[ev_station_v1["State"]=='WA']

In [33]:
ev_station_v1 = ev_station_v1.drop(columns=["Fuel Type Code"])

In [34]:
ev_station_v1.columns

Index(['ID', 'Status Code', 'Station Name', 'Access Code', 'City', 'State',
       'Country', 'Latitude', 'Longitude', 'Open Date', 'EV Pricing'],
      dtype='object')

### 2.3  transforming --------------------------------------

**1. Tranforming status code to user friendly print**

In [35]:
ev_station_v1['Status Code']= ['not_yet_accessible' if x == 'P' else 'temprorally_closed'  if x =='T'else 'open' for x in ev_station_v1['Status Code']]

**2. Adding the quantity column**

In [36]:
ev_station_v1["ev_cs_quantity"]= int(1)

**3. Extracting dollor amount from the price column and transforming to float**

In [37]:
ev_station_v1['EV Pricing'].head()

159     NaN
160    Free
161     NaN
162     NaN
163     NaN
Name: EV Pricing, dtype: object

In [38]:
#define function for conditional formatting
def cond_formatting(x):
    if x ==  'Free':
        return '$0'
    else:
        return x

In [39]:
ev_station_v1['EV Pricing']=ev_station_v1['EV Pricing'].apply(cond_formatting)

In [40]:
ev_station_v1['EV Pricing'].head()

159    NaN
160     $0
161    NaN
162    NaN
163    NaN
Name: EV Pricing, dtype: object

In [41]:
df_price = ev_station_v1['EV Pricing'].str.extract(r'\$(\d+(?:\.\d{2})?)')

In [42]:
print("new column size ", len(df_price))

print("df size ", len(ev_station_v1))

new column size  47086
df size  47086


In [43]:
ev_station_v1['EV Pricing'] = df_price

In [44]:
ev_station_v1['EV Pricing'] = ev_station_v1['EV Pricing'].astype(float)

**4. Adding currency column**

In [45]:
ev_station_v1['currency']= "USD"

**5. Adding PK column**

In [46]:
ev_station_v1 =ev_station_v1.reset_index(drop=True)
ev_station_v1['ev_cs_dim_id'] =  ev_station_v1.index+1

**6. Adding SK Column**

In [47]:
ev_station_v1 =ev_station_v1.reset_index(drop=True)
ev_station_v1['sk_ev_dim_id'] =  ev_station_v1.index+1

**7. Adding SCD columns:**

In [48]:
ev_station_v1["row_effective_date"]= pd.to_datetime("04/21/2023")
ev_station_v1["row_expiration_date"]= pd.Series(dtype='datetime64[ns]')
ev_station_v1["row_status"]="active"

**8. Tranforming columns**

In [49]:
ev_station_v1.columns

Index(['ID', 'Status Code', 'Station Name', 'Access Code', 'City', 'State',
       'Country', 'Latitude', 'Longitude', 'Open Date', 'EV Pricing',
       'ev_cs_quantity', 'currency', 'ev_cs_dim_id', 'sk_ev_dim_id',
       'row_effective_date', 'row_expiration_date', 'row_status'],
      dtype='object')

In [50]:
ev_station_v1.rename(columns={'ID': 'charging_station_id', 
                          'Status Code': 'status_code', 
                          'Status Code': 'status_code', 
                          'EV Pricing': 'ev_price',
                          'Latitude': 'latitude',
                          'Longitude': 'longitude',
                          'Access Code' : 'access_code',
                          'Open Date' : 'activity_date',
                          'City': 'city',
                          'Country': 'country',
                          'State': 'state',
                          'Model': 'vehicle_model',
                          'quantity': 'vehicle_pop_quantity',
                          }, inplace=True)

**9. Transforming activity_date column to date type**

In [51]:
ev_station_v1["activity_date"]= ev_station_v1["activity_date"].astype('datetime64[D]')

In [52]:
ev_station_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47086 entries, 0 to 47085
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   charging_station_id  47086 non-null  int64         
 1   status_code          47086 non-null  object        
 2   Station Name         47086 non-null  object        
 3   access_code          47086 non-null  object        
 4   city                 47086 non-null  object        
 5   state                47086 non-null  object        
 6   country              47086 non-null  object        
 7   latitude             47086 non-null  float64       
 8   longitude            47086 non-null  float64       
 9   activity_date        45791 non-null  datetime64[ns]
 10  ev_price             13376 non-null  float64       
 11  ev_cs_quantity       47086 non-null  int64         
 12  currency             47086 non-null  object        
 13  ev_cs_dim_id         47086 non-

------------------- 

# 3. combining two data frames to one df 

we need to merge two data frame 
we need to check if the sizes are staied the same

In [53]:
print(ev_station_v1.shape)
print(df_ev_pop.shape)

(47086, 18)
(121976, 15)


In [54]:
print('result will have ', ((18-6)+(15-6)+6+1), 'columns')#+1 for the _merge column

result will have  28 columns


In [55]:
df_ev_pop.columns

Index(['vehicle_vin', 'dol_vehicle_id', 'vehicle_make', 'vehicle_model',
       'city', 'state', 'activity_date', 'latitude', 'longitude', 'country',
       'current_reg_status', 'vehicle_pop_quantity', 'ev_car_pop_id',
       'prev_reg_status', 'reg_sta_updated_at'],
      dtype='object')

In [56]:
ev_station_v1.columns

Index(['charging_station_id', 'status_code', 'Station Name', 'access_code',
       'city', 'state', 'country', 'latitude', 'longitude', 'activity_date',
       'ev_price', 'ev_cs_quantity', 'currency', 'ev_cs_dim_id',
       'sk_ev_dim_id', 'row_effective_date', 'row_expiration_date',
       'row_status'],
      dtype='object')

In [57]:
result = pd.merge(ev_station_v1,
                  df_ev_pop,
                  on = ['latitude', 'longitude','city', 'state','country', 'activity_date'],
                  how='outer',
                  indicator=True
)

In [58]:
result.shape

(169062, 28)

# 4 Siplitting data frame 

# 5.Creating the Location data frame 

1. extract the location data 

In [59]:
location_df= result.groupby(['latitude', 'longitude','city', 'state', 'country'], as_index=False).size()
location_df.head()

,latitude,longitude,city,state,country,size
0,-159.71288,21.96605,Kekaha,HI,US,1
1,-158.00833,21.31431,Ewa Beach,HI,US,1
2,-157.94001,21.34183,Honolulu,HI,US,1
3,-157.93084,21.38013,Aiea,HI,US,1
4,-147.72213,64.84527,Fairbanks,AK,US,1


In [61]:
location_df_distinct_city = result.groupby(['city', 'state', 'country'], as_index=False).size()

2. Add region data 

In [62]:
def get_region(state):
    regions = {
        'AL': 'South',
        'AK': 'West',
        'AZ': 'West',
        'AR': 'South',
        'CA': 'West',
        'CO': 'West',
        'CT': 'Northeast',
        'DE': 'South',
        'FL': 'South',
        'GA': 'South',
        'HI': 'West',
        'ID': 'West',
        'IL': 'Midwest',
        'IN': 'Midwest',
        'IA': 'Midwest',
        'KS': 'Midwest',
        'KY': 'South',
        'LA': 'South',
        'ME': 'Northeast',
        'MD': 'South',
        'MA': 'Northeast',
        'MI': 'Midwest',
        'MN': 'Midwest',
        'MS': 'South',
        'MO': 'Midwest',
        'MT': 'West',
        'NE': 'Midwest',
        'NV': 'West',
        'NH': 'Northeast',
        'NJ': 'Northeast',
        'NM': 'West',
        'NY': 'Northeast',
        'NC': 'South',
        'ND': 'Midwest',
        'OH': 'Midwest',
        'OK': 'South',
        'OR': 'West',
        'PA': 'Northeast',
        'RI': 'Northeast',
        'SC': 'South',
        'SD': 'Midwest',
        'TN': 'South',
        'TX': 'South',
        'UT': 'West',
        'VT': 'Northeast',
        'VA': 'South',
        'WA': 'West',
        'WV': 'South',
        'WI': 'Midwest',
        'WY': 'West',
    }
    return regions.get(state, None)


In [63]:
location_df['region']= location_df['state'].apply(get_region)

In [64]:
location_df.head()

,latitude,longitude,city,state,country,size,region
0,-159.71288,21.96605,Kekaha,HI,US,1,West
1,-158.00833,21.31431,Ewa Beach,HI,US,1,West
2,-157.94001,21.34183,Honolulu,HI,US,1,West
3,-157.93084,21.38013,Aiea,HI,US,1,West
4,-147.72213,64.84527,Fairbanks,AK,US,1,West


In [ ]:
location_df = location_df.drop(columns=['size'])

In [77]:
location_df.head()

,latitude,longitude,city,state,country,region
0,-159.71288,21.96605,Kekaha,HI,US,West
1,-158.00833,21.31431,Ewa Beach,HI,US,West
2,-157.94001,21.34183,Honolulu,HI,US,West
3,-157.93084,21.38013,Aiea,HI,US,West
4,-147.72213,64.84527,Fairbanks,AK,US,West


**3. create pk column**

In [78]:
result_loca = pd.merge(result, location_df, on=['latitude', 'longitude', 'city', 'state', 'country'])
result_loca.head()

,charging_station_id,status_code,Station Name,access_code,city,state,country,latitude,longitude,activity_date,...,dol_vehicle_id,vehicle_make,vehicle_model,current_reg_status,vehicle_pop_quantity,ev_car_pop_id,prev_reg_status,reg_sta_updated_at,_merge,region
0,1517.0,open,LADWP - Truesdale Center,private,Sun Valley,CA,US,34.248319,-118.387971,1999-10-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only,West
1,1519.0,open,LADWP - West LA District Office,private,Los Angeles,CA,US,34.052542,-118.448504,2020-02-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only,West
2,1520.0,temprorally_closed,Southern California Edison - Rosemead Office B...,private,Rosemead,CA,US,34.050745,-118.081014,2011-08-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only,West
3,75450.0,temprorally_closed,Southern California Edison - Rosemead Office B...,private,Rosemead,CA,US,34.050745,-118.081014,2016-03-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only,West
4,1523.0,open,Los Angeles Convention Center,public,Los Angeles,CA,US,34.040539,-118.271387,1995-08-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only,West


In [ ]:
# ev_charging_stations_dim_1 = result.loc[result['_merge']=='left_only']
# ev_pop_dim_1 = result.loc[result['_merge']=='right_only']
# print(len(ev_station_v1))
# print(len(ev_charging_stations_dim_1))
# print(len(df_ev_pop))
# print(len(ev_pop_dim_1))
# import numpy as np


# len(np.unique(result[['City', 'State']].values))

In [ ]:
# query example 
# result['dv_pop_quantity']=result['dv_pop_quantity'].fillna(0)
# result['quantity']=result['quantity'].fillna(0)
# result['quantity'].unique()

# summary = result.groupby(['City', 'State', 'Country']).agg({'dv_pop_quantity': 'sum','quantity': 'sum' })
# summary

---------------------------

## splitting the data 

In [ ]:
result.head()

i can create fk for all NaN values and maybe the logic can be if all NaN values okay but then check  

In [ ]:
result.columns

In [ ]:
result['Latitude'] = result['Latitude'].astype(str)
result['Longitude'] = result['Longitude'].astype(str)

In [ ]:
len(result)

In [ ]:
len(result)- len(np.unique(result[['Latitude', 'Longitude','City', 'State']].values))

In [ ]:
len(np.unique(result[['Latitude', 'Longitude','City', 'State', 'Country']].values))

In [ ]:
np.unique(result[['Latitude', 'Longitude','City', 'State', 'Country']].values)

In [ ]:
result =result.reset_index(drop=True)
result['ev_char_fact_id']=  result.index+1

In [ ]:
location_df= result.groupby(['Latitude', 'Longitude','City', 'State', 'Country'], as_index=False).size()
location_df.head()


In [ ]:
location_df = location_df.reset_index(drop=True)

In [ ]:
location_df['location_id'] = location_df.index+1
location_df.head()

-----------------------  --------------------------------

## creating and loading the data 